In [1]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from mlcog import tuning, risk_strat, metrics

In [2]:
ling_train = pd.read_pickle('../data/features/ling_train.pkl')
ling_test = pd.read_pickle('../data/features/ling_test.pkl')

X = ling_train['data']
y = ling_train['label']
X_train_2d = np.stack(X.values)
# Normalization
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train_2d)

feature_abbr = 'cv_ling'  
model_abbr = 'rf'
filename = f"10fcv_{model_abbr}.pkl"
file_path = os.path.join("../data/cv_eval/", feature_abbr, filename)
if os.path.exists(file_path):
    best_model = joblib.load(file_path)

# Assuming best_model is already defined and loaded
results_df = tuning.crossval_predict_with_folds(best_model, X_scaled_train, y)
results_df

,prob,fold,y_true
0,0.414190,3,0
1,0.268214,0,0
2,0.519063,2,0
3,0.528136,3,0
4,0.302866,2,0
...,...,...,...
161,0.596000,4,1
162,0.554976,1,1
163,0.544595,3,1
164,0.883762,4,1


In [3]:
results_df.prob.min(), results_df.prob.max()

(0.10423809523809524, 0.9435714285714286)

In [4]:
cv_probs=results_df.copy()

In [5]:
b = risk_strat.calc_strat_stats(y, cv_probs.prob,resolution=0.1, ignore_amber=True)
b.head()

,green,amber,sens,spec,J,roc_auc,proportion_amber
0,0.15,0.75,0.960000,1.0,0.960000,1.000000,0.837349
1,0.25,0.75,0.923077,1.0,0.923077,0.950000,0.783133
2,0.15,0.85,0.900000,1.0,0.900000,1.000000,0.927711
3,0.25,0.85,0.818182,1.0,0.818182,0.881818,0.873494
4,0.35,0.75,0.727273,1.0,0.727273,0.874126,0.644578


In [6]:
b_filter = b[b['proportion_amber'] < 0.5]
b_filter

,green,amber,sens,spec,J,roc_auc,proportion_amber
6,0.45,0.75,0.558140,1.000000,0.558140,0.795753,0.463855
8,0.45,0.65,0.641509,0.867925,0.509434,0.799573,0.361446
10,0.45,0.55,0.736111,0.754098,0.490209,0.794627,0.198795
11,0.35,0.55,0.854839,0.634146,0.488985,0.820614,0.379518


#### Calc metrics after risk stratification

In [10]:
lingprobs = pd.read_pickle('../data/test_eval_probs/ling_probs_classif.pkl')
lingprobs = lingprobs['Random Forest']
lingprobs

[array([0.24378571, 0.5242619 , 0.60224603, 0.83431746, 0.24952381,
        0.34054762, 0.5516746 , 0.16938095, 0.54938095, 0.71452381,
        0.43909524, 0.44961111, 0.59985714, 0.19633333, 0.65466667,
        0.83136508, 0.55935714, 0.25355556, 0.24871429, 0.40209524,
        0.17602381, 0.84638095, 0.5132381 , 0.85795238, 0.69538095,
        0.58938095, 0.39034921, 0.68904762, 0.38628571, 0.54380952,
        0.77684921, 0.30096825, 0.61783333, 0.42660317, 0.48219048,
        0.6377619 , 0.30633333, 0.30989683, 0.3994127 , 0.35953968,
        0.53088889, 0.5074127 , 0.33202381, 0.89990476, 0.27163492,
        0.12519048, 0.36547619, 0.393     , 0.52159524, 0.35722222,
        0.70088889, 0.553     , 0.186     , 0.36026984, 0.80166667,
        0.27247619, 0.29590476, 0.829     , 0.44177778, 0.70854762,
        0.45446032, 0.32738095, 0.47414286, 0.40652381, 0.38407937,
        0.11785714, 0.30519048, 0.67712698, 0.1588254 , 0.49387302,
        0.85557143]),
 array([0.39078571, 0.79  

In [8]:
y_true_test = ling_test['label']

In [11]:
recalls, specs, aucs, accuracies = metrics.calc_metrics_folds_bootstrap_probabilities(lingprobs, y_true_test, 0.45, 0.65)
accuracy_result = metrics.calculate_metrics_with_ci(accuracies)
recall_result = metrics.calculate_metrics_with_ci(recalls)
spec_result = metrics.calculate_metrics_with_ci(specs)
auc_results = metrics.calculate_metrics_with_ci(aucs)
print("Sensitiviy:", recall_result)
print("Specificity:", spec_result)
print("AUC:", auc_results)
print("Accuracy:", accuracy_result)

Sensitiviy: 67.6 (62.1 - 73.2)
Specificity: 96.7 (93.3 - 100.1)
AUC: 88.7 (86.2 - 91.2)
Accuracy: 83.6 (80.6 - 86.5)


On pilot dataset:

In [12]:
pilot = pd.read_pickle('../data/features/pilot.pkl')
y_true_pilot = pilot['label']
pilot_probs = pd.read_pickle('../data/test_eval_probs/pilot_probs_classif.pkl')
pilot_probs = pilot_probs['Random Forest']

recalls, specs, aucs, accuracies = metrics.calc_metrics_folds_bootstrap_probabilities(pilot_probs, y_true_pilot, 0.45, 0.65)
accuracy_result = metrics.calculate_metrics_with_ci(accuracies)
recall_result = metrics.calculate_metrics_with_ci(recalls)
spec_result = metrics.calculate_metrics_with_ci(specs)
auc_results = metrics.calculate_metrics_with_ci(aucs)
print("Sensitiviy:", recall_result)
print("Specificity:", spec_result)
print("AUC:", auc_results)
print("Accuracy:", accuracy_result)

Sensitiviy: 53.0 (37.4 - 68.7)
Specificity: 73.5 (55.4 - 91.6)
AUC: 67.3 (61.4 - 73.1)
Accuracy: 60.0 (51.5 - 68.5)
